In [1]:
from  selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
import time
import pandas as pd

In [2]:

driver_path = "./chromedriver"

browser = webdriver.Chrome(executable_path=driver_path)
main_url = "https://www.yapikredi.com.tr/memnuniyetiniz-icin-buradayiz/sikca-sorulan-sorular/"

In [3]:
SubGroups = []

In [4]:
def getUrls(main_url):
    browser.get(main_url)
    group_URLs = []
    groupsDiv = browser.find_element_by_class_name("sss-categories")
    print(groupsDiv.text)
    all_children = groupsDiv.find_elements_by_tag_name("a")
        
    for i in range(len(all_children)):
        url = all_children[i].get_attribute("href")
        group_URLs.append(url)
    
    for each in group_URLs:
        print(each)
        
    return group_URLs

In [5]:
def getSubUrls(groupUrl):
    browser.get(groupUrl)
    subgroup_Urls = []
    subgroupsDiv = browser.find_element_by_class_name("gm")
    print(subgroupsDiv.text)
    SubGroups.append(subgroupsDiv.text.split("\n"))
    sub_all_children = subgroupsDiv.find_elements_by_tag_name("li")

    subgroup_Urls =[]
    for i in range(len(sub_all_children)):
        element = sub_all_children[i].find_element_by_tag_name("a")
        url = element.get_attribute("href")
        subgroup_Urls.append(url)

    for each in subgroup_Urls:
        print(each)
    
    return subgroup_Urls

In [6]:
def getNumPages(url):
    browser.get(url)
    pager = browser.find_element_by_id("ctl00_ContentPlaceHolder1_ctrlFaq_pgrFaqsBottom")
    pages = pager.find_elements_by_tag_name("li")
    num_pages = len(pages)
    #single page there is no pager 
    if num_pages == 0:
        num_pages = 1
        
    return num_pages


In [7]:
def collect_Page_Data(url, pageId):
    isNotAttached = True
    data = []
    while isNotAttached:
        try:
            browser.get(url) 
            
            singlePage = False
            try:
                pager = browser.find_element_by_id("ctl00_ContentPlaceHolder1_ctrlFaq_pgrFaqsBottom")
                page = pager.find_element_by_link_text(str(pageId))
            except NoSuchElementException:
                singlePage = True
                print("Single page url no pager")
                
            if not singlePage:       
                page.click()
                print("I have clicked ",page.text)
                
           
            print("I am waiting for 5 secs")
            time.sleep(5)
            
            faqs = browser.find_elements_by_class_name("answer-item")
            for f in faqs:
                qna = "\n".join(f.text.split("\n"))
                data.append(qna)
            
            
            isNotAttached = False
        except StaleElementReferenceException:
            print("WARNING !! Exception in clickPage()")
    #print(data)
    return data
    

In [8]:
def collect_url_data(url):
    all_qnas = []
    num_pages = getNumPages(url)
    print("Number of pages : ", num_pages)
    
    for i in range(num_pages):
        isAttached = False
        while not isAttached:
            try:
                qnas = []
                qnas = collect_Page_Data(url, i+1)
                isAttached = True
                
            except StaleElementReferenceException:
                print("Warning")             
                
        print("attached")
        all_qnas.append(qnas)
        
    return all_qnas
    

In [10]:
dataset = []
Urls = getUrls(main_url)

for idx_u in range(len(Urls)):
    subUrls = getSubUrls(Urls[idx_u])
    for idx_su in range(len(subUrls)):
        data = collect_url_data(subUrls[idx_su])
        tup = (idx_u, idx_su, data)
        dataset.append(tup)
        

Sınırsız Bankacılık Kanalları
Kartlar ve Üye İşyeri
Mevduat ve Yatırım Ürünleri
Bireysel Krediler
Bankacılık Hizmet ve Paketleri
https://www.yapikredi.com.tr/memnuniyetiniz-icin-buradayiz/sikca-sorulan-sorular/sinirsiz-bankacilik-kanallari/6777
https://www.yapikredi.com.tr/memnuniyetiniz-icin-buradayiz/sikca-sorulan-sorular/kartlar-ve-uye-isyeri/6603
https://www.yapikredi.com.tr/memnuniyetiniz-icin-buradayiz/sikca-sorulan-sorular/mevduat-ve-yatirim-urunleri/6579
https://www.yapikredi.com.tr/memnuniyetiniz-icin-buradayiz/sikca-sorulan-sorular/bireysel-krediler/6497
https://www.yapikredi.com.tr/memnuniyetiniz-icin-buradayiz/sikca-sorulan-sorular/bankacilik-hizmet-ve-paketleri/6474
Web Formları
Mobil Bankacılık
İnternet Şubesi
Güvenlik Ürünleri
ATM
https://www.yapikredi.com.tr/memnuniyetiniz-icin-buradayiz/sikca-sorulan-sorular/sinirsiz-bankacilik-kanallari/web-formlari/52348
https://www.yapikredi.com.tr/memnuniyetiniz-icin-buradayiz/sikca-sorulan-sorular/sinirsiz-bankacilik-kanallari/mob

In [14]:
df = pd.DataFrame(dataset, columns =['GroupId', 'SubGroupId', 'Q&A'])

In [12]:
df

,GroupId,SubGroupId,Q&A
0,0,0,[[Web sitesi üzerinden yaptığım başvuruyu nası...
1,0,1,[[ATM’den Akıllı Rehber kayıtlarıma ulaşıp işl...
2,0,2,[[İnternet/mobil şube şifremi belirlerken nele...
3,0,3,"[[Kurumsal İnternet/Mobil Şube üyesiyim, sim k..."
4,0,4,[[Sesli işlem menüsünden hangi işlemleri yapab...
5,1,0,[[Yapı Kredi POS’larından (Sanal POS dahil) Dö...
6,1,1,"[[Kefilli Play öğrenci kartı, Gelir elde etmey..."
7,1,2,[[Play TLcard ile hangi işlemler yapılabilir?\...
8,2,0,[[Nasıl vadesiz hesap açabilirim?\nBankamızda ...
9,2,1,[[Hesap İşletim Ücreti Nedir?\nHesap İşletim Ü...


In [32]:
SubGroups = []
Urls = getUrls(main_url)

for idx_u in range(len(Urls)):
    subUrls = getSubUrls(Urls[idx_u])

Sınırsız Bankacılık Kanalları
Kartlar ve Üye İşyeri
Mevduat ve Yatırım Ürünleri
Bireysel Krediler
Bankacılık Hizmet ve Paketleri
https://www.yapikredi.com.tr/memnuniyetiniz-icin-buradayiz/sikca-sorulan-sorular/sinirsiz-bankacilik-kanallari/6777
https://www.yapikredi.com.tr/memnuniyetiniz-icin-buradayiz/sikca-sorulan-sorular/kartlar-ve-uye-isyeri/6603
https://www.yapikredi.com.tr/memnuniyetiniz-icin-buradayiz/sikca-sorulan-sorular/mevduat-ve-yatirim-urunleri/6579
https://www.yapikredi.com.tr/memnuniyetiniz-icin-buradayiz/sikca-sorulan-sorular/bireysel-krediler/6497
https://www.yapikredi.com.tr/memnuniyetiniz-icin-buradayiz/sikca-sorulan-sorular/bankacilik-hizmet-ve-paketleri/6474
Web Formları
Mobil Bankacılık
İnternet Şubesi
Güvenlik Ürünleri
ATM
https://www.yapikredi.com.tr/memnuniyetiniz-icin-buradayiz/sikca-sorulan-sorular/sinirsiz-bankacilik-kanallari/web-formlari/52348
https://www.yapikredi.com.tr/memnuniyetiniz-icin-buradayiz/sikca-sorulan-sorular/sinirsiz-bankacilik-kanallari/mob

In [33]:
SubGroups

[['Web Formları',
  'Mobil Bankacılık',
  'İnternet Şubesi',
  'Güvenlik Ürünleri',
  'ATM'],
 ['Üye İşyeri', 'Kredi Kartları', 'Banka Kartları'],
 ['Vadesiz Hesap', 'Genel', 'Vadeli Hesap', 'Altın Hesabı'],
 ['Konut', 'Esnek Hesap', 'Taşıt', 'Bireysel İhtiyaç'],
 ['SWIFT', 'Düzenli Ödeme']]

In [34]:
import itertools

categories = list(itertools.chain.from_iterable(SubGroups))

categories

['Web Formları',
 'Mobil Bankacılık',
 'İnternet Şubesi',
 'Güvenlik Ürünleri',
 'ATM',
 'Üye İşyeri',
 'Kredi Kartları',
 'Banka Kartları',
 'Vadesiz Hesap',
 'Genel',
 'Vadeli Hesap',
 'Altın Hesabı',
 'Konut',
 'Esnek Hesap',
 'Taşıt',
 'Bireysel İhtiyaç',
 'SWIFT',
 'Düzenli Ödeme']

In [35]:
df["Categories"] = categories
df

,GroupId,SubGroupId,Q&A,Categories
0,0,0,[[Web sitesi üzerinden yaptığım başvuruyu nası...,Web Formları
1,0,1,[[ATM’den Akıllı Rehber kayıtlarıma ulaşıp işl...,Mobil Bankacılık
2,0,2,[[İnternet/mobil şube şifremi belirlerken nele...,İnternet Şubesi
3,0,3,"[[Kurumsal İnternet/Mobil Şube üyesiyim, sim k...",Güvenlik Ürünleri
4,0,4,[[Sesli işlem menüsünden hangi işlemleri yapab...,ATM
5,1,0,[[Yapı Kredi POS’larından (Sanal POS dahil) Dö...,Üye İşyeri
6,1,1,"[[Kefilli Play öğrenci kartı, Gelir elde etmey...",Kredi Kartları
7,1,2,[[Play TLcard ile hangi işlemler yapılabilir?\...,Banka Kartları
8,2,0,[[Nasıl vadesiz hesap açabilirim?\nBankamızda ...,Vadesiz Hesap
9,2,1,[[Hesap İşletim Ücreti Nedir?\nHesap İşletim Ü...,Genel


In [38]:
for subgroup in df["Q&A"]:
    for page in subgroup:
        for qna in page:
            print(qna)
            print()
            

Web sitesi üzerinden yaptığım başvuruyu nasıl takip edebilirim?
Kredi kartı başvurunuzu web sitesinden Kredi Kartı Başvuru Sorgulama adımından hemen sorgulayabilirsniz. Başvurunuz varsa hemen sorgulamak için tıklayınız.
Bu bilgi size yardımcı oldu mu? EVET HAYIR
YUKARI

Kredi kartı başvurumu nereden yapabilirim?
Türkiye'nin en geniş kart portföyü olan World'e sahip olmak için şubeye gitmeden internetten, SMS ile veya Yapı Kredi Telefon Bankacılığı üzerinden başvuru yapabilirsiniz. Kredi kartına hemen başvurmak için tıklayınız.
Diğer başvuru kanalları hakkında detaylı bilgi için tıklayınız.
Bu bilgi size yardımcı oldu mu? EVET HAYIR
YUKARI

Web sitesi üzerinden yabancı uyruklu müşteriler şifre belirleyebilir mi?
Evet, yabancı uyruklu müşterilerimiz bankamızda kayıtlı Yabancı Kimlik Numaralarını kullanarak şifre belirleyebilirler.
Bu bilgi size yardımcı oldu mu? EVET HAYIR
YUKARI

Web sitesinden sadece yeni kredi kartı için mi şifre belirleyebilirim?
Hayır, sadece yeni kartınız için deği

In [42]:
for sg in dataset:
    print(sg[0:2], categories[dataset.index(sg)] )

(0, 0) Web Formları
(0, 1) Mobil Bankacılık
(0, 2) İnternet Şubesi
(0, 3) Güvenlik Ürünleri
(0, 4) ATM
(1, 0) Üye İşyeri
(1, 1) Kredi Kartları
(1, 2) Banka Kartları
(2, 0) Vadesiz Hesap
(2, 1) Genel
(2, 2) Vadeli Hesap
(2, 3) Altın Hesabı
(3, 0) Konut
(3, 1) Esnek Hesap
(3, 2) Taşıt
(3, 3) Bireysel İhtiyaç
(4, 0) SWIFT
(4, 1) Düzenli Ödeme


In [40]:
for sg in dataset:
    for pgId in range(len(sg[2])):
        print(sg[0:2],pgId)

(0, 0) 0
(0, 0) 1
(0, 0) 2
(0, 1) 0
(0, 2) 0
(0, 2) 1
(0, 2) 2
(0, 2) 3
(0, 2) 4
(0, 3) 0
(0, 3) 1
(0, 3) 2
(0, 3) 3
(0, 3) 4
(0, 4) 0
(0, 4) 1
(0, 4) 2
(0, 4) 3
(1, 0) 0
(1, 0) 1
(1, 0) 2
(1, 0) 3
(1, 1) 0
(1, 1) 1
(1, 1) 2
(1, 1) 3
(1, 1) 4
(1, 1) 5
(1, 1) 6
(1, 1) 7
(1, 1) 8
(1, 1) 9
(1, 1) 10
(1, 1) 11
(1, 2) 0
(2, 0) 0
(2, 1) 0
(2, 2) 0
(2, 3) 0
(3, 0) 0
(3, 0) 1
(3, 1) 0
(3, 2) 0
(3, 2) 1
(3, 3) 0
(3, 3) 1
(4, 0) 0
(4, 1) 0


In [44]:
data = []
for sg in dataset:
    for pgId in range(len(sg[2])):
        #print(sg[0:2],pgId, sg[2][pgId])
        
        for qna in sg[2][pgId]:
            #print(qna)
            list_qna = qna.split("\n")
            list_qna.pop()
            list_qna.pop()
            processed_qna = "\n".join(list_qna)
            print(sg[0:2],processed_qna)
            new_tup = (sg[0],sg[1],categories[dataset.index(sg)] ,processed_qna)
            data.append(new_tup)
            print("***********")

(0, 0) Web sitesi üzerinden yaptığım başvuruyu nasıl takip edebilirim?
Kredi kartı başvurunuzu web sitesinden Kredi Kartı Başvuru Sorgulama adımından hemen sorgulayabilirsniz. Başvurunuz varsa hemen sorgulamak için tıklayınız.
***********
(0, 0) Kredi kartı başvurumu nereden yapabilirim?
Türkiye'nin en geniş kart portföyü olan World'e sahip olmak için şubeye gitmeden internetten, SMS ile veya Yapı Kredi Telefon Bankacılığı üzerinden başvuru yapabilirsiniz. Kredi kartına hemen başvurmak için tıklayınız.
Diğer başvuru kanalları hakkında detaylı bilgi için tıklayınız.
***********
(0, 0) Web sitesi üzerinden yabancı uyruklu müşteriler şifre belirleyebilir mi?
Evet, yabancı uyruklu müşterilerimiz bankamızda kayıtlı Yabancı Kimlik Numaralarını kullanarak şifre belirleyebilirler.
***********
(0, 0) Web sitesinden sadece yeni kredi kartı için mi şifre belirleyebilirim?
Hayır, sadece yeni kartınız için değil, şifresini unuttuğunuz ya da değiştirmek istediğiniz kartınız için de şifre belirleme i

In [45]:
df = pd.DataFrame(data, columns =['GroupId', 'SubGroupId', 'Categories','Q&A'])

In [46]:
df

,GroupId,SubGroupId,Categories,Q&A
0,0,0,Web Formları,Web sitesi üzerinden yaptığım başvuruyu nasıl ...
1,0,0,Web Formları,Kredi kartı başvurumu nereden yapabilirim?\nTü...
2,0,0,Web Formları,Web sitesi üzerinden yabancı uyruklu müşterile...
3,0,0,Web Formları,Web sitesinden sadece yeni kredi kartı için mi...
4,0,0,Web Formları,Kredi kartı şifremi nasıl belirleyebilirim?\nB...
5,0,0,Web Formları,Aynı gayrimenkul ilanı için 1 den fazla başvur...
6,0,0,Web Formları,Başka kişiler tarafından verilen teklifler hak...
7,0,0,Web Formları,Burada belirtilen fiyatın üzerine KDV eklenece...
8,0,0,Web Formları,Gayrimenkuller ile ilgili daha detaylı bilgi a...
9,0,0,Web Formları,Vadeli teklif vermek veya ödemenin taksitlere ...


In [47]:
# testUniqueness()
qnaList = df["Q&A"]
qnaSet = set(qnaList)

print(len(qnaSet),len(qnaList))

367 389


In [48]:
dup = df[df.duplicated(keep = "first")]
dup

,GroupId,SubGroupId,Categories,Q&A
197,1,1,Kredi Kartları,Öğrenci kredi kartı limit arttırımı nasıl yapı...
285,1,1,Kredi Kartları,Faydalanabileceğim sigortalar nelerdir?\n1) Yu...
287,1,1,Kredi Kartları,"Kefilli Play öğrenci kartı, Gelir elde etmeye ..."
288,1,1,Kredi Kartları,"Opet Worldcard nedir?\nOpet Worldcard, Opet ve..."
289,1,1,Kredi Kartları,Opet Worldcard’a nasıl başvurulur?\nOpet World...
290,1,1,Kredi Kartları,Kredi kartından nakit çekim yapıldığında ne ka...
291,1,1,Kredi Kartları,Yapı Kredi Seyahat Hattı nedir?\nYapı Kredi Se...
292,1,1,Kredi Kartları,Kredi kartı faizi nedir? Nasıl hesaplanır?\nAl...
293,1,1,Kredi Kartları,adios kart sahipleri yurtdışında hangi avantaj...
294,1,1,Kredi Kartları,Alışverişlerde Worldcard’ı nasıl kullanırsınız...


In [49]:
print(df.size)
print(df.shape)
print(len(df))

1556
(389, 4)
389


In [50]:
df = df.drop_duplicates( keep = "first" ,inplace = False)

In [51]:
print(df.shape)

(367, 4)


In [61]:
df.reset_index()

,index,GroupId,SubGroupId,Categories,Q&A
0,0,0,0,Web Formları,Web sitesi üzerinden yaptığım başvuruyu nasıl ...
1,1,0,0,Web Formları,Kredi kartı başvurumu nereden yapabilirim?\nTü...
2,2,0,0,Web Formları,Web sitesi üzerinden yabancı uyruklu müşterile...
3,3,0,0,Web Formları,Web sitesinden sadece yeni kredi kartı için mi...
4,4,0,0,Web Formları,Kredi kartı şifremi nasıl belirleyebilirim?\nB...
5,5,0,0,Web Formları,Aynı gayrimenkul ilanı için 1 den fazla başvur...
6,6,0,0,Web Formları,Başka kişiler tarafından verilen teklifler hak...
7,7,0,0,Web Formları,Burada belirtilen fiyatın üzerine KDV eklenece...
8,8,0,0,Web Formları,Gayrimenkuller ile ilgili daha detaylı bilgi a...
9,9,0,0,Web Formları,Vadeli teklif vermek veya ödemenin taksitlere ...


In [67]:
df = df.reset_index(drop=True)
df

,GroupId,SubGroupId,Categories,Q&A
0,0,0,Web Formları,Web sitesi üzerinden yaptığım başvuruyu nasıl ...
1,0,0,Web Formları,Kredi kartı başvurumu nereden yapabilirim?\nTü...
2,0,0,Web Formları,Web sitesi üzerinden yabancı uyruklu müşterile...
3,0,0,Web Formları,Web sitesinden sadece yeni kredi kartı için mi...
4,0,0,Web Formları,Kredi kartı şifremi nasıl belirleyebilirim?\nB...
5,0,0,Web Formları,Aynı gayrimenkul ilanı için 1 den fazla başvur...
6,0,0,Web Formları,Başka kişiler tarafından verilen teklifler hak...
7,0,0,Web Formları,Burada belirtilen fiyatın üzerine KDV eklenece...
8,0,0,Web Formları,Gayrimenkuller ile ilgili daha detaylı bilgi a...
9,0,0,Web Formları,Vadeli teklif vermek veya ödemenin taksitlere ...


In [68]:
dup = df[df.duplicated(keep = "first")]
dup

,GroupId,SubGroupId,Categories,Q&A


In [69]:
df

,GroupId,SubGroupId,Categories,Q&A
0,0,0,Web Formları,Web sitesi üzerinden yaptığım başvuruyu nasıl ...
1,0,0,Web Formları,Kredi kartı başvurumu nereden yapabilirim?\nTü...
2,0,0,Web Formları,Web sitesi üzerinden yabancı uyruklu müşterile...
3,0,0,Web Formları,Web sitesinden sadece yeni kredi kartı için mi...
4,0,0,Web Formları,Kredi kartı şifremi nasıl belirleyebilirim?\nB...
5,0,0,Web Formları,Aynı gayrimenkul ilanı için 1 den fazla başvur...
6,0,0,Web Formları,Başka kişiler tarafından verilen teklifler hak...
7,0,0,Web Formları,Burada belirtilen fiyatın üzerine KDV eklenece...
8,0,0,Web Formları,Gayrimenkuller ile ilgili daha detaylı bilgi a...
9,0,0,Web Formları,Vadeli teklif vermek veya ödemenin taksitlere ...


In [72]:
export_csv = df.to_csv ("/Users/cankutcoskun/Desktop/TextClassification/labeled_qnas.csv", header=True) 
export_xlsx = df.to_excel("/Users/cankutcoskun/Desktop/TextClassification/labeled_qnas.xlsx", header=True) 